# 4.Analicen la linealidad del circuito, realizando adquisiciones a una frecuencia determinada y variando la amplitud de modulación.
- ¿Como elegirían la frecuencia de modulación?
- Analice la relación Señal/Ruido
- ¿Hay algún desfasaje entre la señal de modulación y lo que miden? ¿Como evitarían que eso afecte al proceso de medición?

Arrancamos chequeando puertos y definiendo el lock-in

In [ ]:
import visa

rm = visa.ResourceManager()

print(rm.list_resources()) #Preguntamos los puertos

#Vemos que devuelve el puerto "GPIB0::11::INSTR" que es el que queremos, pero aún así lo chequeamos

inst = rm.open_resource("GPIB0::11::INSTR") #Inicializamos comunicación con ese puerto
print(inst.query("*IDN?")) #Le preguntamos que instrumento esta conectado al puerto

#Debería devolver que es el lock-in SR830

Definimos lock-in

In [ ]:
from lockin import SR830

config = {
          'lockin_addr': 'GPIB0::11::INSTR', 
          'medicion_modo' : 0, #Canal A single-ended
          'display_modo' : 'RT', #Nos muestra R y Tita
          'sens' : 12, #Sensibilidad 20mV
          'slope' : 2, #Orden del filtro 6dB/oct
          't_int' : 8, #Frecuencia de corte 10Hz 
          'ref_intern' : True, #Él mismo genera la referencia
          'ref_freq' : 1e4, #Con frecuencia 80Hz
          'ref_v' : 0.1, #Con amplitud 0.1V
          }

lock = SR830(config)

Ahora hacemos un barrido para distintos voltajes. Usamos de frecuencia de referencia 1000Hz

In [ ]:
import numpy as np
from time import sleep

vmax = 4.95
amplitudes = np.arange(0.05, vmax, 0.05)

sens = 17
f_corte = 8
f_mod = 1e3
orden = 3
lock.setSensibility(sens) 
lock.setIntegrationTime(f_corte)
lock.setFreqReferencia(f_mod)
lock.setFilterSlope(orden)
sleep(5)

Xs = []
Ys = []
for amp in amplitudes:
    lock.setVoltReferencia(amp)
    sleep(0.5)
    X, Y = lock.getMedicion("XY")
    Xs.append(X)
    Ys.append(Y)
    
np.savetxt(f"actividad_4/Xs_p4_fc{f_corte}_s{sens}_o{orden}_fm{f_mod}_vmax{vmax}.txt",
           Xs, delimiter=",")
np.savetxt(f"actividad_4/Ys_p4_fc{f_corte}_s{sens}_o{orden}_fm{f_mod}_vmax{vmax}.txt",
           Ys, delimiter=",") 